<a href="https://colab.research.google.com/github/shahshawaiz/artwork-style-prediction/blob/master/artwork_style.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# # activate kaggle, mount grdive, download/unzip dataset
# import os
# os.environ['KAGGLE_USERNAME'] = "shahshawaiz" # username from the json file
# os.environ['KAGGLE_KEY'] = "476e4fb10eef6d9da79358d4df0db985" # key from the json file

from google.colab import drive
drive.mount('/content/drive')

# files = ['train_1.zip', 'train_2.zip', 'train_3.zip', 'train_4.zip', 'train_5.zip', 'train_6.zip', 'train_7.zip', 'train_8.zip', 'train_9.zip']
# for path in files:
#   path_dir = path.strip('.zip')
#   path_gdrive = '/content/drive/My Drive/Colab Notebooks/dataset/wikiart/train'

#   !kaggle competitions download -c painter-by-numbers -f "$path"
#   !unzip -qq "$path"
#   !mv -n "$path_dir"/ "$path_gdrive"/
#   !rm -rf "$path" "$path_dir"

# # drive.flush_and_unmount()   

# # # # extract zip
# # # # !unzip '/content/drive/My Drive/university/s3/lfi/project/data/train_1.zip'

# # from tensorflow.python.client import device_lib
# # print(device_lib.list_local_devices())

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [0]:
# pkgs installation
# !pip install tensorflow h5py pyyaml

In [3]:
# imports
import numpy as np
import pandas as pd
import os
import time
from keras.applications.resnet50 import ResNet50
from keras.preprocessing import image
from keras.layers import GlobalAveragePooling2D, Dense, Dropout,Activation,Flatten
from keras.callbacks import ModelCheckpoint

from keras.applications.vgg16 import preprocess_input
from keras.layers import Input
from keras.models import Model, load_model
from keras.utils import np_utils
from sklearn.utils import shuffle
from sklearn.model_selection import train_test_split

from typing import Any, Dict, List, Optional, Pattern, Set, Tuple

Using TensorFlow backend.


In [0]:
# set env
MODEL_VERSION = 'a'
PATH = '/content/drive/My Drive/Colab Notebooks/dataset/wikiart'
PATH_IMAGES = PATH + '/train'
PATH_METADATA = PATH + '/all_data_info.csv'
PATH_MODEL = PATH + '/models/m-' + MODEL_VERSION + '.h5'
PATH_CHECKPOINT = PATH + '/checkpoints/cp-' + MODEL_VERSION + '.ckpt'
CHECKPOINT_PERIOD = 3
SLICE_SIZE = 5000
NUM_CLASSES = 25

In [0]:
#@title
# read dataset
def read_data():
  img_list = []
  img_missing_list = []

  # metadata
  metadata = pd.read_csv(PATH_METADATA)
  print(metadata.head())
  top_n_classes = metadata.groupby(['style']).style.value_counts().nlargest(NUM_CLASSES).index.get_level_values(0).values.tolist() # get first n classes
  metadata = metadata[metadata['style'].isin(top_n_classes)] # filter metadata w.r.t classes
  
  images_path = metadata['new_filename'].sample(n=SLICE_SIZE, random_state=1).values.tolist()
  metadata = metadata[metadata['new_filename'].isin(images_path)] # filter metadata w.r.t slice size

  train_dirs = os.listdir(PATH_IMAGES)
  # train_dirs = ['train_1', 'train_2']

  # read images
  for filename in images_path:
    image_missing = True

    for train_dir in train_dirs:
        img_path = PATH_IMAGES + '/'+ train_dir + '/' + filename	
        try:
          img = image.load_img(img_path, target_size=(224, 224))
          x = image.img_to_array(img)
          x = np.expand_dims(x, axis=0)
          x = preprocess_input(x)
          img_list.append(x)

          image_missing = False
        except OSError as e:
          pass
      
    if image_missing:
      print("File not found:", img_path)
      img_missing_list.append(filename)

  # convert to float32, fix shape
  images = np.array(img_list)
  images = images.astype('float32')
  images = np.rollaxis(images,1,0)
  
  # remove missing files from metadata
  metadata = metadata[~metadata['new_filename'].isin(img_missing_list)] 

  print("Missing files count: ", len(img_missing_list))  
  return images[0], metadata

def factorize(metadata):
	metadata['style_code'] = metadata['style'].astype('category')
	metadata['style_code'] = pd.factorize(metadata.style_code)[0]
	
	return metadata

def get_categories(labels):
	return np_utils.to_categorical(labels, NUM_CLASSES)
	
def shuffle_data(images, Y):
  x, y = shuffle(images, Y, random_state=2)
  return train_test_split(x, y, test_size=0.2, random_state=2)

In [0]:
def resnet_init():
  #image_input = Input(shape=(224, 224, 3))
  model = ResNet50(weights='imagenet', include_top=False)
  model.summary()  

  return model

In [0]:
def resnet_retrain_classifier(model):
  last_layer = model.output
  # add a global spatial average pooling layer
  x = GlobalAveragePooling2D()(last_layer)
  # add fully-connected & dropout layers
  x = Dense(512, activation='relu',name='fc-1')(x)
  x = Dropout(0.5)(x)
  x = Dense(256, activation='relu',name='fc-2')(x)
  x = Dropout(0.5)(x)
  # a softmax layer for n classes
  out = Dense(NUM_CLASSES, activation='softmax',name='output_layer')(x)  

  return out

In [0]:
def get_retrained_resnet(model, out):
  # # this is the model we will train
  model = Model(inputs=model.input, outputs=out)
  
  for layer in model.layers[:-6]:
    layer.trainable = False

  model.layers[-1].trainable
  model.compile(loss='categorical_crossentropy',optimizer='adam',metrics=['accuracy'])  
  
  model.summary()

  return model

In [0]:
def set_checkpoints():
  checkpoint_dir = os.path.dirname(PATH_CHECKPOINT)

  # create checkpount
  return ModelCheckpoint(filepath=PATH_CHECKPOINT, save_weights_only=True, verbose=1, period=CHECKPOINT_PERIOD)

In [0]:
def get_saved_model():
  model = None
  
  try:
    print("Model found at: ", PATH_MODEL)
  except OSError as e:
    print("Model doesn't exist")
    
  return model

def set_weights(model): 
  try:
    model.load_weights(PATH_CHECKPOINT)
    print("Checkpoint found at: ", PATH_CHECKPOINT)
  except OSError as e:
    print("Checkpoint doesn't exist")

  return model

In [0]:
def model_create():

	model = get_saved_model()
	if model is None:
		model = resnet_init()
		out = resnet_retrain_classifier(model)
		model = get_retrained_resnet(model, out)
	
	model = set_weights(model)
 
	return model

In [0]:
def model_fit(
    model, X_train, y_train, X_test, y_test, cp_callback
):
  t=time.time()
  hist = model.fit(
      X_train, y_train, batch_size=32, epochs=12, verbose=1, validation_data=(X_test, y_test), callbacks=[cp_callback]
  )
  print('Training time: %s' % (t - time.time()))
  (loss, accuracy) = model.evaluate(X_test, y_test, batch_size=10, verbose=1)

  print("[INFO] loss={:.4f}, accuracy: {:.4f}%".format(loss,accuracy * 100))  

  return model, hist

In [0]:
def model_visiualize(hist):
  
  import matplotlib.pyplot as plt
  # visualizing losses and accuracy
  train_loss=hist.history['loss']
  val_loss=hist.history['val_loss']
  train_acc=hist.history['acc']
  val_acc=hist.history['val_acc']
  xc=range(12)

  plt.figure(1,figsize=(7,5))
  plt.plot(xc,train_loss)
  plt.plot(xc,val_loss)
  plt.xlabel('num of Epochs')
  plt.ylabel('loss')
  plt.title('train_loss vs val_loss')
  plt.grid(True)
  plt.legend(['train','val'])
  #print plt.style.available # use bmh, classic,ggplot for big pictures
  plt.style.use(['classic'])

  plt.figure(2,figsize=(7,5))
  plt.plot(xc,train_acc)
  plt.plot(xc,val_acc)
  plt.xlabel('num of Epochs')
  plt.ylabel('accuracy')
  plt.title('train_acc vs val_acc')
  plt.grid(True)
  plt.legend(['train','val'],loc=4)
  #print plt.style.available # use bmh, classic,ggplot for big pictures
  plt.style.use(['classic'])  

In [0]:
#@title
images, metadata = read_data()

           artist    date     genre  ...  artist_group  in_train  new_filename
0  Barnett Newman  1955.0  abstract  ...    train_only      True    102257.jpg
1  Barnett Newman  1950.0  abstract  ...    train_only      True     75232.jpg
2     kiri nichol  2013.0       NaN  ...     test_only     False     32145.jpg
3     kiri nichol  2013.0       NaN  ...     test_only     False     20304.jpg
4     kiri nichol  2013.0       NaN  ...     test_only     False       836.jpg

[5 rows x 12 columns]
File not found: /content/drive/My Drive/Colab Notebooks/dataset/wikiart/train/train_4/48247.jpg
File not found: /content/drive/My Drive/Colab Notebooks/dataset/wikiart/train/train_4/101708.jpg
File not found: /content/drive/My Drive/Colab Notebooks/dataset/wikiart/train/train_4/16559.jpg
File not found: /content/drive/My Drive/Colab Notebooks/dataset/wikiart/train/train_4/37121.jpg
File not found: /content/drive/My Drive/Colab Notebooks/dataset/wikiart/train/train_4/49926.jpg
File not found: /conte

In [0]:
metadata = factorize(metadata)
Y = get_categories(metadata.style_code)
X_train, X_test, y_train, y_test = shuffle_data(images, Y)

model = model_create()
cp_callback = set_checkpoints()

model, hist = model_fit(
    model, X_train, y_train, X_test, y_test, cp_callback
)
model.save(PATH_MODEL)

model_visiualize(hist)